# Introduction

<div class="alert alert-warning">
<font color=black>

**What?** Exporting to ONNX

</font>
</div>

# What is ONNX?

<div class="alert alert-info">
<font color=black>

- In this recipe, we will look at exporting PyTorch models into the Open Neural Network Exchange (ONNX), which provides an open source format for both deep learning and traditional machine learning models. 
- It defines an extensible computation graph model, as well as built-in operators and standard data types.

</font>
</div>

# Import modules

In [3]:
import onnx
import onnxruntime
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(64 * 4 * 4, 512)
        self.linear2 = nn.Linear(512, 10) 
        self.dropout = nn.Dropout(p=0.3)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [5]:
model = CNN()

In [8]:
model.load_state_dict(torch.load("cifar10.pth"))

FileNotFoundError: [Errno 2] No such file or directory: 'cifar10.pth'

In [7]:
model.eval()

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [6]:
x = torch.randn(1, 3, 32, 32, requires_grad=True)

In [7]:
model_out = model(x)

In [8]:
model_out

tensor([[ 3.5831, 10.1674, -4.0835, -4.4179, -4.0980, -5.8097, -2.0841, -1.6725,
          1.0503,  8.2375]], grad_fn=<AddmmBackward>)

In [9]:
torch.onnx.export(model,
                 x,
                 "cifar.onnx",
                 export_params=True,
                 opset_version=10,         
                 do_constant_folding=True, 
                 input_names = ['input'],  
                 output_names = ['output'],
                 dynamic_axes={'input' : {0 : 'batch_size'},
                               'output' : {0 : 'batch_size'}})

In [10]:
onnx_model = onnx.load("cifar.onnx")

In [11]:
onnx.checker.check_model(onnx_model)

In [12]:
ort_session = onnxruntime.InferenceSession("cifar.onnx")

In [13]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [14]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

In [15]:
np.testing.assert_allclose(to_numpy(model_out), ort_outs[0], rtol=1e-03, atol=1e-05)

# References

<div class="alert alert-warning">
<font color=black>

- Jibin Mathew, PyTorch Artificial Intelligence Fundamentals

</font>
</div>